In [ ]:
#%pip install numpy scikit-learn

### Vectorización de texto y modelo de clasificación Naïve Bayes con el dataset 20 newsgroups

In [1]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.metrics import f1_score
import random

# 20newsgroups por ser un dataset clásico de NLP ya viene incluido y formateado
# en sklearn
from sklearn.datasets import fetch_20newsgroups
import numpy as np

## Carga de datos

In [4]:
# cargamos los datos (ya separados de forma predeterminada en train y test)
newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))
newsgroups_test = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'))

## Vectorización

In [2]:
# instanciamos un vectorizador
# ver diferentes parámetros de instanciación en la documentación de sklearn https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html
tfidfvect = TfidfVectorizer()

In [5]:
# en el atributo `data` accedemos al texto
print(newsgroups_train.data[0])

I was wondering if anyone out there could enlighten me on this car I saw
the other day. It was a 2-door sports car, looked to be from the late 60s/
early 70s. It was called a Bricklin. The doors were really small. In addition,
the front bumper was separate from the rest of the body. This is 
all I know. If anyone can tellme a model name, engine specs, years
of production, where this car is made, history, or whatever info you
have on this funky looking car, please e-mail.


In [6]:
# con la interfaz habitual de sklearn podemos fitear el vectorizador
# (obtener el vocabulario y calcular el vector IDF)
# y transformar directamente los datos
X_train = tfidfvect.fit_transform(newsgroups_train.data)
# `X_train` la podemos denominar como la matriz documento-término

In [7]:
# recordar que las vectorizaciones por conteos son esparsas
# por ello sklearn convenientemente devuelve los vectores de documentos
# como matrices esparsas
print(type(X_train))
print(f'shape: {X_train.shape}')
print(f'Cantidad de documentos: {X_train.shape[0]}')
print(f'Tamaño del vocabulario (dimensionalidad de los vectores): {X_train.shape[1]}')

<class 'scipy.sparse._csr.csr_matrix'>
shape: (11314, 101631)
Cantidad de documentos: 11314
Tamaño del vocabulario (dimensionalidad de los vectores): 101631


In [8]:
# una vez fiteado el vectorizador, podemos acceder a atributos como el vocabulario
# aprendido. Es un diccionario que va de términos a índices.
# El índice es la posición en el vector de documento.
tfidfvect.vocabulary_['car']

25775

In [9]:
# es muy útil tener el diccionario opuesto que va de índices a términos
idx2word = {v: k for k,v in tfidfvect.vocabulary_.items()}

In [10]:
# en `y_train` guardamos los targets que son enteros
y_train = newsgroups_train.target
y_train[:10]

array([ 7,  4,  4,  1, 14, 16, 13,  3,  2,  4])

In [11]:
# hay 20 clases correspondientes a los 20 grupos de noticias
print(f'clases {np.unique(newsgroups_test.target)}')
newsgroups_test.target_names

clases [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]


['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

## Similaridad de documentos

In [12]:
# Veamos similaridad de documentos. Tomemos algún documento
idx = 4811
print(newsgroups_train.data[idx])

THE WHITE HOUSE

                  Office of the Press Secretary
                   (Pittsburgh, Pennslyvania)
______________________________________________________________
For Immediate Release                         April 17, 1993     

             
                  RADIO ADDRESS TO THE NATION 
                        BY THE PRESIDENT
             
                Pittsburgh International Airport
                    Pittsburgh, Pennsylvania
             
             
10:06 A.M. EDT
             
             
             THE PRESIDENT:  Good morning.  My voice is coming to
you this morning through the facilities of the oldest radio
station in America, KDKA in Pittsburgh.  I'm visiting the city to
meet personally with citizens here to discuss my plans for jobs,
health care and the economy.  But I wanted first to do my weekly
broadcast with the American people. 
             
             I'm told this station first broadcast in 1920 when
it reported that year's presidential elec

In [13]:
# midamos la similaridad coseno con todos los documentos de train
cossim = cosine_similarity(X_train[idx], X_train)[0]

In [14]:
# podemos ver los valores de similaridad ordenados de mayor a menos
np.sort(cossim)[::-1]

array([1.        , 0.70930477, 0.67474953, ..., 0.        , 0.        ,
       0.        ])

In [15]:
# y a qué documentos corresponden
np.argsort(cossim)[::-1]

array([4811, 6635, 4253, ..., 9019, 9016, 8748])

In [16]:
# los 5 documentos más similares:
mostsim = np.argsort(cossim)[::-1][1:6]

In [17]:
# el documento original pertenece a la clase:
newsgroups_train.target_names[y_train[idx]]

'talk.politics.misc'

In [18]:
# y los 5 más similares son de las clases:
for i in mostsim:
  print(newsgroups_train.target_names[y_train[i]])

talk.politics.misc
talk.politics.misc
talk.politics.misc
talk.politics.misc
talk.politics.misc


### Modelo de clasificación Naïve Bayes

In [19]:
# es muy fácil instanciar un modelo de clasificación Naïve Bayes y entrenarlo con sklearn
clf = MultinomialNB()
clf.fit(X_train, y_train)

MultinomialNB()

In [20]:
# con nuestro vectorizador ya fiteado en train, vectorizamos los textos
# del conjunto de test
X_test = tfidfvect.transform(newsgroups_test.data)
y_test = newsgroups_test.target
y_pred =  clf.predict(X_test)

In [21]:
# el F1-score es una metrica adecuada para reportar desempeño de modelos de claificación
# es robusta al desbalance de clases. El promediado 'macro' es el promedio de los
# F1-score de cada clase. El promedio 'micro' es equivalente a la accuracy que no
# es una buena métrica cuando los datasets son desbalanceados
f1_score(y_test, y_pred, average='macro')

0.5854345727938506

### Consigna del desafío 1

**Cada experimento realizado debe estar acompañado de una explicación o interpretación de lo observado.**

**1**. Vectorizar documentos. Tomar 5 documentos al azar y medir similaridad con el resto de los documentos.
Estudiar los 5 documentos más similares de cada uno analizar si tiene sentido
la similaridad según el contenido del texto y la etiqueta de clasificación.

**2**. Construir un modelo de clasificación por prototipos (tipo zero-shot). Clasificar los documentos de un conjunto de test comparando cada uno con todos los de entrenamiento y asignar la clase al label del documento del conjunto de entrenamiento con mayor similaridad.

**3**. Entrenar modelos de clasificación Naïve Bayes para maximizar el desempeño de clasificación
(f1-score macro) en el conjunto de datos de test. Considerar cambiar parámteros
de instanciación del vectorizador y los modelos y probar modelos de Naïve Bayes Multinomial
y ComplementNB.

**NO cambiar el hiperparámetro ngram_range de los vectorizadores**.

**4**. Transponer la matriz documento-término. De esa manera se obtiene una matriz
término-documento que puede ser interpretada como una colección de vectorización de palabras.
Estudiar ahora similaridad entre palabras tomando 5 palabras y estudiando sus 5 más similares.

**Elegir las palabras MANUALMENTE para evitar la aparición de términos poco interpretables**.


----
## Respuestas:

**1**. Vectorizar documentos. Tomar 5 documentos al azar y medir similaridad con el resto de los documentos.
Estudiar los 5 documentos más similares de cada uno analizar si tiene sentido
la similaridad según el contenido del texto y la etiqueta de clasificación.

In [22]:
idx1 = random.randint(0, X_train.shape[0])
print(f'Se va a trabajar con el documento ubicado en la posicion {idx1}: \n')
print(newsgroups_train.data[idx1])
print('\n\nPertenece a la clase ',newsgroups_train.target_names[y_train[idx1]])
# Similaridad coseno con todos los documentos de train
cossim = cosine_similarity(X_train[idx1], X_train)[0]
# los 5 documentos más similares:
mostsim = np.argsort(cossim)[::-1][1:6]
# y los 5 más similares son de las clases:
print("Los 5 documentos mas similares son: ",mostsim)
for i in mostsim:
  print(f'&&&&&&& Documento {i}: {newsgroups_train.data[i]}')
  print(f'Clase: {newsgroups_train.target_names[y_train[i]]}')


Se va a trabajar con el documento ubicado en la posicion 9995: 

I just noticed that my halogen table lamp runs off 12 Volts.
The big thinngy that plugs into the wall says 12 Volts DC,  20mA

The question is: Can I trickle charge the battery on my CB650
with it?

I don't know the rating of the battery, but it is a factory
intalled one. 


Thanks,
Sanjay

-- 
   '81 CB650 						DoD #1224


Pertenece a la clase  rec.motorcycles
Los 5 documentos mas similares son:  [1241  431 1079 7791  821]
&&&&&&& Documento 1241: :>Curtis JAckson pens...
:>
:>"MSF course...$140"
:
:Shyah!
:
:The one here only costs $35!
:
:(Izzat a deal or what?! :)


They are free in Philadelphia....   :-)

-- 
   '81 CB650 						DoD #1224
Clase: rec.motorcycles
&&&&&&& Documento 431: I have a problem with the battery on my '83 Honda CB650 NightHawk.
Every week or so it is dead and I have to recharge it. I ride the bike 
every day, the battery is new and the charging system was checked 
thoroughly and it seems fine. My

**2**. Construir un modelo de clasificación por prototipos (tipo zero-shot). Clasificar los documentos de un conjunto de test comparando cada uno con todos los de entrenamiento y asignar la clase al label del documento del conjunto de entrenamiento con mayor similaridad.

In [23]:
# 1. seleccionar un ejemplo del dataset test
# 2. calcular similitud coseno con todo el dataset train
# 3. seleccionar el mas similar
# 4. seleccionar la etiqueta de ese ejemplo mas similar de train

In [ ]:
def clasificador_baseline(indice_test):
  idxtest = indice_test
  cossim = cosine_similarity(X_test[idxtest], X_train)[0]
  # El documento más similar es:
  mostsim = np.argsort(cossim)[::-1][1]
  categoria_predict = y_train[mostsim]
  return(categoria_predict)


predicciones = [clasificador_baseline(idx) for idx in range(X_test.shape[0])]


In [ ]:
f1_score(y_test, predicciones, average='macro')

**3**. Entrenar modelos de clasificación Naïve Bayes para maximizar el desempeño de clasificación
(f1-score macro) en el conjunto de datos de test. Considerar cambiar parámteros
de instanciación del vectorizador y los modelos y probar modelos de Naïve Bayes Multinomial
y ComplementNB.

In [ ]:
# es muy fácil instanciar un modelo de clasificación Naïve Bayes y entrenarlo con sklearn
clf = MultinomialNB()
clf.fit(X_train, y_train)
# con nuestro vectorizador ya fiteado en train, vectorizamos los textos
# del conjunto de test
X_test = tfidfvect.transform(newsgroups_test.data)
y_test = newsgroups_test.target
y_pred =  clf.predict(X_test)
# es una buena métrica cuando los datasets son desbalanceados
f1_score(y_test, y_pred, average='macro')

0.5854345727938506

In [ ]:

from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score

# Definir modelos
models = {
    "MultinomialNB": MultinomialNB(),
    "ComplementNB": ComplementNB()
}

# Parámetros para búsqueda
param_grid = {
    "MultinomialNB": {"alpha": [0.1, 0.5, 1.0]},
    "ComplementNB": {"alpha": [0.1, 0.5, 1.0]}
}

# Evaluar cada modelo
results = {}
for name, model in models.items():
    grid = GridSearchCV(model, param_grid[name], scoring='f1_macro', cv=5)
    grid.fit(X_train, y_train)
    best_model = grid.best_estimator_
    y_pred = best_model.predict(X_test)
    f1 = f1_score(y_test, y_pred, average='macro')
    results[name] = {
        "best_params": grid.best_params_,
        "f1_macro": f1
    }

# Mostrar resultados
for model_name, result in results.items():
    print(f"Modelo: {model_name}")
    print(f"Mejores parámetros: {result['best_params']}")
    print(f"F1-score macro en test: {result['f1_macro']:.4f}\\n")


Modelo: MultinomialNB
Mejores parámetros: {'alpha': 0.1}
F1-score macro en test: 0.6565\n
Modelo: ComplementNB
Mejores parámetros: {'alpha': 0.1}
F1-score macro en test: 0.6954\n


**4**. Transponer la matriz documento-término. De esa manera se obtiene una matriz
término-documento que puede ser interpretada como una colección de vectorización de palabras.
Estudiar ahora similaridad entre palabras tomando 5 palabras y estudiando sus 5 más similares.

**Elegir las palabras MANUALMENTE para evitar la aparición de términos poco interpretables**.

In [ ]:
X_transposed = X_train.transpose()

In [ ]:
# Obtener nombres de términos
terms = tfidfvect.get_feature_names_out()

# Seleccionar 5 términos para estudiar
selected_terms = ["programming", "information"]

# Obtener índices de los términos seleccionados
term_indices = [np.where(terms == term)[0][0] for term in selected_terms]

# Calcular similaridad coseno entre vectores de términos
similarities = cosine_similarity(X_transposed)

# Mostrar las 5 palabras más similares para cada término seleccionado
for idx in term_indices:
    term = terms[idx]
    sim_scores = similarities[idx]
    # Obtener índices de las 5 más similares (excluyendo el término en sí)
    top_indices = np.argsort(sim_scores)[::-1][1:6]
    top_terms = [terms[i] for i in top_indices]
    print(f"Términos más similares a '{term}': {top_terms}\n")
